In [1]:
# Load libraries
# !pip install -r /work/NLP_IMDb_Exam/requirements.txt
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sentence_transformers import SentenceTransformer

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-22 19:32:03.024853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734892323.050846   17681 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734892323.058844   17681 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 19:32:03.090122: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [2]:
# Read embeddings from CSV
data_path = '../Data/IMDB_Embeddings_Base.csv'
emb_full=pd.read_csv(data_path)
emb_full = emb_full.drop(columns=['Unnamed: 0']) # Drop index collumn
emb_full

,0,1,2,3,4,5,6,7,8,9,...,376,377,378,379,380,381,382,383,rating,review
0,-0.081544,0.011703,-0.009561,0.073821,0.037057,0.041061,-0.103449,0.060129,0.092972,-0.016048,...,-0.023995,-0.037069,-0.101025,0.020209,-0.034976,-0.056466,-0.093876,-0.052097,7.2,This movie is full of suspense. It makes you g...
1,-0.031219,-0.040882,-0.050346,0.026394,0.096147,0.035325,-0.035926,0.038864,0.026956,-0.008654,...,-0.040147,-0.066848,-0.060416,0.053252,-0.032352,-0.008767,-0.003945,0.011570,7.2,I'll leave this review fairly concise. <br/><b...
2,-0.043770,-0.009958,-0.014713,0.105553,0.023357,0.071526,-0.042314,0.035394,0.055924,-0.002175,...,-0.045357,-0.002683,-0.082446,0.021947,-0.024405,0.000253,-0.058909,-0.017371,7.2,I give the 5/10 out of the credit I owe to the...
3,0.016111,-0.021361,0.027286,0.035094,0.076533,-0.025168,-0.032712,0.038052,0.069695,0.031988,...,0.015142,0.066612,-0.021369,0.029857,0.002516,-0.012402,-0.052657,0.015130,7.2,"First of all, I must say that I was expecting ..."
4,0.006157,-0.034961,-0.019405,0.023993,0.040031,0.106513,-0.019048,-0.057289,0.071020,0.034495,...,-0.043846,0.073074,0.007909,-0.027694,-0.017272,0.013925,-0.101928,0.002226,7.2,I've always loved movies with strong atmospher...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835597,-0.070385,0.011580,0.018101,-0.114973,-0.026786,0.035987,0.057268,0.085646,0.018540,0.017948,...,0.042943,0.040848,-0.067290,0.006085,-0.021538,-0.006587,-0.016786,-0.001022,7.7,I found it most interesting to read the commen...
835598,-0.111276,0.045682,-0.079033,-0.000299,-0.000158,0.043070,-0.013706,-0.037212,-0.048332,0.051722,...,0.019284,0.088586,-0.044019,0.025105,0.105852,-0.066657,-0.053592,-0.015313,7.7,Zulu is a film I can watch over and over again...
835599,-0.111976,0.086066,-0.050779,-0.035097,0.009463,0.052782,0.020150,-0.069127,0.023358,0.000965,...,-0.003177,0.087524,-0.000856,0.077876,0.056882,0.032958,-0.062826,-0.002790,7.7,"This is a fantastic film of a true, historical..."
835600,-0.070247,0.021456,0.023077,-0.007591,-0.026042,0.044647,-0.056316,-0.021921,0.001819,0.033487,...,-0.016904,0.027902,-0.041811,0.026836,0.093079,-0.019669,-0.134161,0.014316,7.7,(I was getting chills just casting my vote for...


In [3]:
IMDb_dataframe = pd.read_csv('/work/NLP_IMDb_Exam/Data/review_dataframe.csv')
IMDb_dataframe = IMDb_dataframe[IMDb_dataframe['rating'] != "Null"] # Remove nulls
IMDb_dataframe = IMDb_dataframe.reset_index(drop=True)
IMDb_dataframe

,movie_title,year,rating,title,review,average_rating
0,10 Cloverfield Lane,2016,7,Unfortunately the ending ruined an otherwise g...,This movie is full of suspense. It makes you g...,7.2
1,10 Cloverfield Lane,2016,5,...oh dear Abrams. Again.\n,I'll leave this review fairly concise. <br/><b...,7.2
2,10 Cloverfield Lane,2016,5,"Fantastic, gripping, thoroughly enjoyable, unt...",I give the 5/10 out of the credit I owe to the...,7.2
3,10 Cloverfield Lane,2016,5,Hmmm...\n,"First of all, I must say that I was expecting ...",7.2
4,10 Cloverfield Lane,2016,7,Slow building & plot alternating claustrophobi...,I've always loved movies with strong atmospher...,7.2
...,...,...,...,...,...,...
835597,Zulu,1964,8,hard to beat for its genre\n,I found it most interesting to read the commen...,7.7
835598,Zulu,1964,8,One of the great war films\n,Zulu is a film I can watch over and over again...,7.7
835599,Zulu,1964,10,Great battle scenes!\n,"This is a fantastic film of a true, historical...",7.7
835600,Zulu,1964,9,"Ye Gods, what a stirring movie!\n",(I was getting chills just casting my vote for...,7.7


In [4]:
from sklearn.model_selection import train_test_split
# Remove rest of unecessary columns before splitting:
emb_full['rating'] = IMDb_dataframe['rating']
split = emb_full.drop(columns=['review'])
split = train_test_split(split, train_size=0.7)
finaleval=split[1]
subset=split[0]

train_features = subset.drop(columns=["rating"]).to_numpy()
train_labels = np.float32(subset['rating'].to_numpy())
eval_features = finaleval.drop(columns=["rating"]).to_numpy()
eval_labels = np.float32(finaleval[['rating']].to_numpy())

# Create a linear regression of entire embedding space:

In [6]:
#!pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 24.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 1.5 MB/s eta 0:00:00ta 0:00:01


In [7]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [8]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.4303209434392463
MSE: 4.732243904123986
Corr: 0.6548844794289174


,True Labels,Predicted Labels
0,9.0,10.718242
1,8.0,8.227584
2,10.0,9.173659
3,10.0,12.287770
4,7.0,8.549114
...,...,...
250676,10.0,10.172106
250677,5.0,7.668089
250678,8.0,5.588066
250679,2.0,3.896066


# Create an ICA model:

In [9]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=20, whiten="arbitrary-variance") # Define ICA
My_ICA_fit = ica.fit_transform(train_features) # Fit ICA to data

In [10]:
# Get estimated mixing matrix
mixMatrixEst = ica.mixing_

# Map embeddings onto ICA-space
train_ICA_features = pd.DataFrame(train_features @ mixMatrixEst)
eval_ICA_features = pd.DataFrame(eval_features @ mixMatrixEst)

In [11]:
# !pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

In [12]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [13]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.4303209434392463
MSE: 4.732243904123986
Corr: 0.6548844794289174


,True Labels,Predicted Labels
0,9.0,10.718242
1,8.0,8.227584
2,10.0,9.173659
3,10.0,12.287770
4,7.0,8.549114
...,...,...
250676,10.0,10.172106
250677,5.0,7.668089
250678,8.0,5.588066
250679,2.0,3.896066
